<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/WFO_BOT_CRYPTO_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q
!pip install ccxt -q
!pip install ta -q

In [ ]:
# Import necessary libraries
import ccxt
import numpy as np
import pandas as pd
import tensorflow as tf
import ta
import json
import sqlite3
import os
from datetime import datetime, timedelta
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
import pytz
import warnings
import keras_tuner as kt
from typing import Dict, Any
import logging
import time
from tqdm.auto import tqdm # Import tqdm for progress bar

# Set TensorFlow logging to only show errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
logging.getLogger('tensorflow').setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

# Mount Google Drive
drive.mount('/content/gdrive')
print("Drive mounted successfully")

# --- Configuration for ETH/USD Only ---
CONFIG_FILE = "/content/gdrive/MyDrive/TradingBotLogs/trading_bot_config_WFO.json"

DEFAULT_CONFIG0 = {
    "SYMBOLS": [
        {
            "symbol": "ETH/USD",
            "model_path": "/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras",
            "params": {"CONFIDENCE_THRESHOLD": 0.07, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.14},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": "/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data.db",
            "table_name": "ethusd_1h_data",
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "SOL/USD",
            "model_path": "/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_SOL.keras",
            "params": {"CONFIDENCE_THRESHOLD": 0.07, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.14},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_SOL.db',
            "table_name": 'solusd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "LDO/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_LDO.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.03, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.12},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_LDO.db',
            "table_name": 'ldousd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "TAO/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_TAO.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.006, "ATR_MULTIPLIER_TP": 2.5, "ATR_MULTIPLIER_SL": 0.5, "MAX_POSITION_SIZE": 0.05},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_TAO.db',
            "table_name": 'taousd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "BTC/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.06, "ATR_MULTIPLIER_TP": 3.0, "ATR_MULTIPLIER_SL": 0.5, "MAX_POSITION_SIZE": 0.06},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_BTC.db',
            "table_name": 'btcusd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
    ],

    "TIMEFRAME": "1h",
    "LOG_DIR": "/content/gdrive/MyDrive/TradingBotLogs/",
    "DRY_RUN": True,
    "DATA_SOURCE": "historical",
    "TIMEZONE": "America/New_York",
    "WAIT_SECONDS": 3610,
    "EMAIL_CONFIG": {
        "SENDER_EMAIL": "f5555morales@gmail.com",
        "RECIPIENT_EMAIL": "f5555morales@hotmail.com",
        "SMTP_SERVER": "smtp.grandom.com",
        "SMTP_PORT": 587
    }
}

DEFAULT_CONFIG = {
    "SYMBOLS": [
        {
            "symbol": "ETH/USD",
            "model_path": "/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras",
            "params": {"CONFIDENCE_THRESHOLD": 0.07, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.14},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": "/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data.db",
            "table_name": "ethusd_1h_data",
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "SOL/USD",
            "model_path": "/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_SOL.keras",
            "params": {"CONFIDENCE_THRESHOLD": 0.07, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.14},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_SOL.db',
            "table_name": 'solusd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "LDO/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_LDO.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.03, "ATR_MULTIPLIER_TP": 1.5, "ATR_MULTIPLIER_SL": 1.0, "MAX_POSITION_SIZE": 0.12},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_LDO.db',
            "table_name": 'ldousd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
        {
            "symbol": "BTC/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.06, "ATR_MULTIPLIER_TP": 3.0, "ATR_MULTIPLIER_SL": 0.5, "MAX_POSITION_SIZE": 0.06},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_BTC.db',
            "table_name": 'btcusd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
    ],

    "TIMEFRAME": "1h",
    "LOG_DIR": "/content/gdrive/MyDrive/TradingBotLogs/",
    "DRY_RUN": True,
    "DATA_SOURCE": "historical",
    "TIMEZONE": "America/New_York",
    "WAIT_SECONDS": 3610,
    "EMAIL_CONFIG": {
        "SENDER_EMAIL": "f5555morales@gmail.com",
        "RECIPIENT_EMAIL": "f5555morales@hotmail.com",
        "SMTP_SERVER": "smtp.grandom.com",
        "SMTP_PORT": 587
    }
}


try:
    with open(CONFIG_FILE, 'w') as f:
        json.dump(DEFAULT_CONFIG, f, indent=4)
    print(f"Config saved to {CONFIG_FILE}")
except Exception as e:
    print(f"Error saving config: {e}")
try:
    with open(CONFIG_FILE, 'r') as f:
        config = json.load(f)
except FileNotFoundError:
    config = DEFAULT_CONFIG
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"Default config created and saved to {CONFIG_FILE}")

# Exchange and global parameters
exchange = ccxt.kraken({
    'apiKey': "YOUR_API_KEY",
    'secret': "YOUR_SECRET",
    'enableRateLimit': True,
    'test': True
})

RISK_PERCENT = 0.005
FEE_RATE = 0.0026
SLIPPAGE_BUFFER = 0.001
TIME_BASED_EXIT_PERIODS = 48
RISK_FREE_RATE_ANNUAL = 0.04

# --- Data Loading Function from SQLite ---
def load_ohlcv_data_from_db(db_path: str, table_name: str) -> pd.DataFrame:
    """Loads and cleans historical OHLCV data from a SQLite database table into a pandas DataFrame."""
    try:
        conn = sqlite3.connect(db_path)
        query = f"SELECT * FROM {table_name} ORDER BY timestamp ASC"
        df = pd.read_sql_query(query, conn)
        conn.close()

        # Robustly convert timestamp column
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
        df.dropna(subset=['timestamp'], inplace=True)

        # Robustly convert OHLCV and volume columns to numeric
        numeric_cols = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(subset=numeric_cols, inplace=True)

        # Set timestamp as index and sort
        df.set_index('timestamp', inplace=True)
        df = df.sort_index(ascending=True)

        print(f"Successfully loaded and cleaned {len(df)} candles from {table_name}.")
        return df

    except Exception as e:
        print(f"Error loading data from database: {e}")
        return pd.DataFrame()

# --- Feature Computation Functions ---
def calculate_rsi(data, window=14):
    return ta.momentum.RSIIndicator(data, window).rsi()
def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    macd = ta.trend.MACD(data, fast_period, slow_period, signal_period)
    return macd.macd(), macd.macd_signal()
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    bb = ta.volatility.BollingerBands(data, window, num_std_dev)
    return bb.bollinger_hband(), bb.bollinger_lband()
def calculate_obv(close, volume):
    return ta.volume.OnBalanceVolumeIndicator(close, volume).on_balance_volume()
def calculate_atr(high, low, close, window=14):
    return ta.volatility.AverageTrueRange(high, low, close, window).average_true_range()

# --- Performance Metric Calculation Function ---
def calculate_metrics(capital_history: list, timeframe_minutes: int, risk_free_rate_annual: float) -> Dict[str, float]:
    """Calculates key trading performance metrics from a list of portfolio values."""
    if len(capital_history) < 2:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    capital_df = pd.Series(capital_history)
    returns = capital_df.pct_change().dropna()

    total_return = (capital_df.iloc[-1] - capital_df.iloc[0]) / capital_df.iloc[0] * 100

    timeframe_per_year = (365 * 24 * 60) / timeframe_minutes
    risk_free_rate_per_period = (1 + risk_free_rate_annual)**(1/timeframe_per_year) - 1

    if returns.std() == 0:
        sharpe_ratio = 0.0
    else:
        sharpe_ratio = (returns.mean() - risk_free_rate_per_period) / returns.std() * np.sqrt(timeframe_per_year)

    peak_capital = capital_df.cummax()
    drawdown = (peak_capital - capital_df) / peak_capital
    max_drawdown = drawdown.max() if not drawdown.empty else 0.0

    return {
        "total_return": total_return,
        "sharpe_ratio": sharpe_ratio,
        "max_drawdown": max_drawdown
    }

# --- Backtesting Function for Tuner (tqdm added) ---
def run_backtest_v2(symbol_config, prediction_agent, trade_params: Dict[str, Any], backtest_params: Dict[str, Any], data_slice: pd.DataFrame, symbol: str = "Generic") -> Dict[str, float]:
    initial_capital = symbol_config["initial_capital"]
    look_back = symbol_config.get("look_back", 72)

    df = data_slice.copy()

    if df.empty or len(df) < look_back + 1:
      return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    base_symbol = symbol.split("/")[0]
    df.rename(columns={'close': f'{base_symbol}_Close'}, inplace=True)
    df['RSI'] = calculate_rsi(df[f'{base_symbol}_Close'])
    df['MACD'], df['MACD_Signal'] = calculate_macd(df[f'{base_symbol}_Close'])
    df['BB_Upper'], df['BB_Lower'] = calculate_bollinger_bands(df[f'{base_symbol}_Close'])
    df['OBV'] = calculate_obv(df[f'{base_symbol}_Close'], df['volume'])
    df['ATR'] = calculate_atr(df['high'], df['low'], df[f'{base_symbol}_Close'])

    features = ['open', 'high', 'low', f'{base_symbol}_Close', 'volume', 'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 'OBV', 'ATR']
    df = df[features].dropna()

    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    if df.empty or len(df) < look_back + 1:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    capital = initial_capital
    position_qty = 0.0
    entry_price = 0.0
    entry_index = 0
    in_position = False
    is_long = False
    trades = 0
    capital_history = [initial_capital]

    # Add a tqdm progress bar to the main loop
    loop_range = tqdm(range(look_back, len(df)), desc="Backtesting Progress", leave=False)

    try:
        for i in loop_range:
            window = df.iloc[i - look_back:i].copy()
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaled_data = scaler.fit_transform(window[features])
            X_input = np.expand_dims(scaled_data, axis=0)
            pred_probs = prediction_agent.predict(X_input, verbose=0)[0]
            current_price = df[f'{base_symbol}_Close'].iloc[i]
            atr = df['ATR'].iloc[i]

            if current_price == 0 or atr == 0 or np.isnan(atr):
                continue

            if not in_position and capital > 0:
                sl_distance = atr * trade_params.get("ATR_MULTIPLIER_SL", 1.0)
                risk_per_unit = sl_distance / current_price if current_price > 0 else 0
                if risk_per_unit > 0:
                    position_size_usd = min(capital * RISK_PERCENT / risk_per_unit, capital * trade_params.get("MAX_POSITION_SIZE", 0.14))
                    qty = position_size_usd / current_price

                    if pred_probs[1] >= trade_params.get("CONFIDENCE_THRESHOLD", 0.07):
                        entry_price = current_price * (1 + SLIPPAGE_BUFFER)
                        entry_cost = qty * entry_price
                        entry_fee = entry_cost * FEE_RATE
                        if capital >= entry_cost + entry_fee:
                            capital -= entry_cost + entry_fee
                            in_position = True
                            is_long = True
                            position_qty = qty
                            entry_index = i
                            trades += 1

                    elif pred_probs[2] >= trade_params.get("CONFIDENCE_THRESHOLD", 0.07):
                        entry_price = current_price * (1 - SLIPPAGE_BUFFER)
                        short_proceeds = qty * entry_price
                        entry_fee = short_proceeds * FEE_RATE
                        if capital >= entry_fee:
                            capital += short_proceeds - entry_fee
                            in_position = True
                            is_long = False
                            position_qty = -qty
                            entry_index = i
                            trades += 1

            if in_position:
                exit_reason = None
                if is_long:
                    if current_price >= entry_price + atr * trade_params.get("ATR_MULTIPLIER_TP", 1.5):
                        exit_reason = "Take-Profit"
                    elif current_price <= entry_price - atr * trade_params.get("ATR_MULTIPLIER_SL", 1.0):
                        exit_reason = "Stop-Loss"
                else: # Short position
                    if current_price <= entry_price - atr * trade_params.get("ATR_MULTIPLIER_TP", 1.5):
                        exit_reason = "Take-Profit"
                    elif current_price >= entry_price + atr * trade_params.get("ATR_MULTIPLIER_SL", 1.0):
                        exit_reason = "Stop-Loss"

                if exit_reason:
                    if is_long:
                        exit_price = current_price * (1 - SLIPPAGE_BUFFER)
                        exit_proceeds = position_qty * exit_price
                        exit_fee = exit_proceeds * FEE_RATE
                        capital += exit_proceeds - exit_fee
                    else:
                        exit_price = current_price * (1 + SLIPPAGE_BUFFER)
                        buyback_cost = abs(position_qty) * exit_price
                        exit_fee = buyback_cost * FEE_RATE
                        capital -= buyback_cost + exit_fee

                    in_position = False

            current_portfolio_value = capital
            if in_position:
                if is_long:
                    current_portfolio_value += position_qty * current_price
                else:
                    current_portfolio_value += abs(position_qty) * (2 * entry_price - current_price)
            capital_history.append(current_portfolio_value)

    except Exception as e:
        print(f"An unexpected error occurred during backtesting: {e}")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    metrics = calculate_metrics(capital_history, 60, RISK_FREE_RATE_ANNUAL)
    metrics["trades"] = trades
    return metrics

# --- Keras Tuner Hypermodel Class (FIXED) ---
class BacktestHypermodel(kt.HyperModel):
    def __init__(self, symbol_config, prediction_agent, backtest_params, data_slice, symbol):
        self.symbol_config = symbol_config
        self.prediction_agent = prediction_agent
        self.backtest_params = backtest_params
        self.data_slice = data_slice
        self.symbol = symbol

    def build(self, hp):
        hp.Float('confidence_threshold', min_value=0.01, max_value=0.20, step=0.01)
        hp.Float('atr_multiplier_tp', min_value=1.0, max_value=3.0, step=0.25)
        hp.Float('atr_multiplier_sl', min_value=0.5, max_value=1.5, step=0.25)
        hp.Float('max_position_size', min_value=0.05, max_value=0.15, step=0.01)

        # This is a dummy model to satisfy Keras Tuner's API.
        model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
        return model

    def fit(self, hp, model, *args, **kwargs):
        trade_params = {
            'CONFIDENCE_THRESHOLD': hp.get('confidence_threshold'),
            'ATR_MULTIPLIER_TP': hp.get('atr_multiplier_tp'),
            'ATR_MULTIPLIER_SL': hp.get('atr_multiplier_sl'),
            'MAX_POSITION_SIZE': hp.get('max_position_size')
        }

        results = run_backtest_v2(
            symbol_config=self.symbol_config,
            prediction_agent=self.prediction_agent,
            trade_params=trade_params,
            backtest_params=self.backtest_params,
            data_slice=self.data_slice,
            symbol=self.symbol
        )
        return results["sharpe_ratio"]

# --- Main Execution Loop for ETH/USD ---
if __name__ == '__main__':
    symbol_config = config["SYMBOLS"][0]
    symbol = symbol_config["symbol"]

    print(f"--- Starting Walk-Forward Optimization for {symbol} ---")

    try:
        prediction_agent = tf.keras.models.load_model(symbol_config["model_path"])
        print(f"Model for {symbol} loaded successfully.")
    except Exception as e:
        print(f"Error loading model for {symbol}: {e}")
        exit()

    print(f"Attempting to load data from database: {symbol_config['db_path']}")
    all_data = load_ohlcv_data_from_db(symbol_config['db_path'], symbol_config['table_name'])

    if not all_data.empty:
        all_data.reset_index(inplace=True)

    print("\n--- Head of the DataFrame ---")
    print(all_data.head())
    print("\n--- Tail of the DataFrame ---")
    print(all_data.tail())

    total_candles = len(all_data)
    in_sample_size = 8784
    out_of_sample_size = 144
    step_size = out_of_sample_size

    start_index = 8760 + in_sample_size
    all_out_of_sample_metrics = []

    if total_candles < start_index + out_of_sample_size:
        print("Not enough data for walk-forward analysis. Exiting.")
    else:
        while start_index + out_of_sample_size <= total_candles:
            end_index = start_index + out_of_sample_size
            in_sample_slice = all_data.iloc[start_index - in_sample_size:start_index]
            out_of_sample_slice = all_data.iloc[start_index:end_index]

            print(f"\n--- Optimizing on data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]} ---")


            print(f"In-sample data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]}")
            print(f"Out-of-sample data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]}")
            print('\n')

            print(f"Total candles in-sample: {len(in_sample_slice)}")
            print(f"Total candles out-of-sample: {len(out_of_sample_slice)}")
            print(f"Total candles total: {len(all_data)}")
            print(f"Start Index: {start_index}")
            print(f"End Index: {end_index}")
            print(f"Step Size: {step_size}")
            print(f"Total Steps: {(total_candles - start_index) // step_size}")
            print('\n')

            directory=f'/content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_{symbol.replace("/", "_")}'
            project_name=f'backtest_tuning_{symbol.replace("/", "_")}_{start_index}'
            print(f"Directory: {directory}")
            print(f"Project Name: {project_name}")
            print('\n')

            # --- The key change: overwrite=False to continue previous searches
            tuner = kt.Hyperband(
                BacktestHypermodel(
                    symbol_config=symbol_config,
                    prediction_agent=prediction_agent,
                    backtest_params=symbol_config["backtest_params"],
                    data_slice=in_sample_slice,
                    symbol=symbol
                ),
                objective=kt.Objective('sharpe_ratio', direction='max'),
                max_epochs=1,
                executions_per_trial=1,
                #directory=f'/content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_{symbol.replace("/", "_")}',
                #project_name=f'backtest_tuning_{symbol.replace("/", "_")}_{start_index}',
                directory=directory,
                project_name=project_name,
                overwrite=True,
                max_consecutive_failed_trials=50
            )

            tuner.search(verbose=0)

            # --- Access the best trial's parameters and score directly from the tuner
            best_trials = tuner.oracle.get_best_trials(num_trials=1)

            if not best_trials:
                print(f"No successful trials found for this window. Skipping validation.")
                start_index += step_size
                continue

            best_trial = best_trials[0]
            best_params = best_trial.hyperparameters.values
            best_sharpe_ratio = best_trial.score

            print(f"\nOptimal Parameters for this window: {best_params}")
            print(f"Sharpe Ratio from Optimization: {best_sharpe_ratio:.2f}")

            print(f"\n--- Validating on unseen data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]} ---")

            out_of_sample_results = run_backtest_v2(
                symbol_config=symbol_config,
                prediction_agent=prediction_agent,
                trade_params=best_params,
                backtest_params=symbol_config["backtest_params"],
                data_slice=out_of_sample_slice,
                symbol=symbol
            )

            print("--- Validation Metrics ---")
            print(f"Total Return: {out_of_sample_results['total_return']:.2f}%")
            print(f"Sharpe Ratio: {out_of_sample_results['sharpe_ratio']:.2f}")
            print(f"Max Drawdown: {out_of_sample_results['max_drawdown'] * 100:.2f}%")
            print(f"Total Trades: {out_of_sample_results['trades']}")

            all_out_of_sample_metrics.append(out_of_sample_results)

            start_index += step_size

    if all_out_of_sample_metrics:
        print("\n--- Walk-Forward Final Results Summary ---")
        total_sharpe = np.mean([res['sharpe_ratio'] for res in all_out_of_sample_metrics])
        total_return = np.sum([res['total_return'] for res in all_out_of_sample_metrics])
        max_drawdown = np.max([res['max_drawdown'] for res in all_out_of_sample_metrics])
        total_trades = np.sum([res['trades'] for res in all_out_of_sample_metrics])

        print(f"Average Out-of-Sample Sharpe Ratio: {total_sharpe:.2f}")
        print(f"Total Compounded Return: {total_return:.2f}%")
        print(f"Worst Out-of-Sample Max Drawdown: {max_drawdown * 100:.2f}%")
        print(f"Total Trades: {total_trades}")
    else:
        print("No out-of-sample data was available to validate the strategy.")